In [266]:
import sys
sys.path.insert(1, '/Users/nimish/Desktop/SHELL/evaluator')

import gc
import itertools
import numpy as np
from evaluator import *

In [46]:
'''
evaluator.py
IMPORTS:

getTurbLoc(TURB_LOC_file_name)

loadPowerCurve(POWER_CURVE_file_name)

binWindResourceData(WIND_DATA_file_name)

searchSorted(lookup, sample_array)

preProcessing(POWER_CURVE)

getAEP(TURB_DIAM, turb_coords, POWER_CURVE, WIND_INST_FREQ,
       N_WIND_INSTANCES, COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t)
       
checkConstraints(turb_coords, TURB_DIAM)

'''

# Turbine Specifications.
# -**-SHOULD NOT BE MODIFIED-**-
TURB_SPECS    =  {
                     'Name': 'Anon Name',
                     'Vendor': 'Anon Vendor',
                     'Type': 'Anon Type',
                     'Dia (m)': 100,
                     'Rotor Area (m2)': 7853,
                     'Hub Height (m)': 100,
                     'Cut-in Wind Speed (m/s)': 3.5,
                     'Cut-out Wind Speed (m/s)': 25,
                     'Rated Wind Speed (m/s)': 15,
                     'Rated Power (MW)': 3
                 }
TURB_DIAM      =  TURB_SPECS['Dia (m)']
TURB_DIAM       =  TURB_DIAM/2

# Load the power curve
POWER_CURVE   =  loadPowerCurve('./../evaluator/power_curve.csv')

# Pass wind data csv file location to function binWindResourceData.
# Retrieve probabilities of wind instance occurence.
WIND_INST_FREQ =  binWindResourceData('./../evaluator/wind_data/wind_data_2007.csv')

# Doing preprocessing to avoid the same repeating calculations. Record
# the required data for calculations. Do that once. Data are set up (shaped)
# to assist vectorization. Used later in function totalAEP.
N_WIND_INSTANCES, COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t = preProcessing(POWER_CURVE)

# check if there is any constraint is violated before we do anything. 
def evalPrint(turbine_coordinates):
    checkConstraints(turbine_coordinates, TURB_DIAM)
    
    print('[INFO] Calculating AEP')
    AEP = getAEP(TURB_DIAM, turbine_coordinates, POWER_CURVE, WIND_INST_FREQ,
                 N_WIND_INSTANCES, COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t)
    print('[INFO] Power produced : ', "%.12f"%(AEP), 'GWh')
    
    
# TEST VALUES
# Turbine x,y coordinates
TEST_COORDS   =  getTurbLoc(r'./../test_locations.csv')

In [47]:
%%time
# SELF TEST
_ = checkConstraints(TEST_COORDS, TURB_DIAM)
print("--------######---------")
evalPrint(TEST_COORDS)
print()

[SUCCESS] perimeter and proximity constraints - SATISFIED
--------######---------
[SUCCESS] perimeter and proximity constraints - SATISFIED
[INFO] Calculating AEP
[INFO] Power produced :  505.450636596680 GWh

CPU times: user 137 ms, sys: 15.5 ms, total: 153 ms
Wall time: 157 ms


## Classes

In [191]:
class DNA:
    def __init__(self):
        self.fitness = 0
        # self.genes = np.random.randint(low=50, high=3950, size=100)
       
    # GENERATE SET OF VALID GENES
    def generate_genes():
        pass
        
    # CALC & STORE FITNESS
    def calc_fitness(self):
        if(checkConstraints_F(self.get_tuples(), TURB_DIAM) == -1):
            return 0
        self.fitness = getAEP(TURB_DIAM, self.get_tuples(), POWER_CURVE, WIND_INST_FREQ, 
                              N_WIND_INSTANCES,COS_DIR, SIN_DIR, WIND_SPEED_STACKED, C_t)
        return self.fitness
    
    # GET DNA AS TUPLE OF COORDINATES
    def get_tuples(self):
        return np.reshape(self.genes, newshape=(50,2))
    
    def __getitem__(self, index):
        print("[ERR] USE d.genes[i]")

In [192]:
class Population:
    def __init__(self, population_size, mutation_rate):
        self.generations = 0
        self.best_DNA = None
        self.best_fitness = 0
        
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        
        self.population = np.array([DNA() for _ in range(self.population_size)])
    
    # CALC & GET FITNESS
    def calc_fitnesses(self):
        return np.array([self.population[i].calc_fitness() for i in range(self.population_size)])
    
    def get_fitnesses(self):
        return np.array([self.population[i].fitness for i in range(self.population_size)])
    
    ##
    
    # GET DNAs
    def get_population_DNA(self):
        return np.array([self.population[i].get_DNA() for i in range(self.population_size)])
        
    def __getitem__(self, index):
        print("[ERR] USE p.population[i]")

In [185]:
x = Population(2, 0.1)

# Main

In [215]:
temp0 = np.array([[300,455],[2392,556]])
temp0

array([[ 300,  455],
       [2392,  556]])

In [221]:
temp1 = np.random.uniform(low=50,high=3950,size=(2)).astype(np.int64)
temp1

array([], shape=(1, 0), dtype=float64)

In [227]:
np.concatenate([np.array([], dtype=np.int64).reshape(0,2), temp0 ,temp1.reshape(1,2)], axis=0)

array([[ 300,  455],
       [2392,  556],
       [2116, 3201]])

In [264]:
def generate():
    items = np.array([], dtype=np.int64).reshape(0,2)
    
    count = 0
    iters = 0
    while(count <= 50):
        temp_tuple = np.random.uniform(low=50,high=3950,size=(1,2)).astype(np.int64)
        temp_items = np.concatenate([items, temp], axis=0) 
        if(checkConstraints_F(temp_items, TURB_DIAM) == 1):
            items = np.concatenate([items, temp], axis=0)
            count += 1
        iters += 1
        if(iters % 1000 == 0):
            print(iters)
    return items

In [265]:
%%time
x = generate()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000


KeyboardInterrupt: 